In [6]:
import pandas as pd
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import csv



class Image:

    def __init__(self) -> None:
        pass   
            
    def get_block(self,image,row,column):
        # img = cv2.imread(image,0)
        ht, wd = image.shape
        ch = 0 
        cw = 0
        nh = ht//row
        nw = wd//column
        ls = []
        m = nh

        if cw == 0:
            cri = image[ch:nh,cw:nw]
            # print(type(cri))
            ls.append(cri)
        n = nw
        for j in range(2,column+1):
            cw = nw
            nw = n*j
            cri = image[ch:nh,cw:nw]
            ls.append(cri) 
        for i in range(0,row):
            if i > 0:
                rc = i+1
                #print(i)
                ch = nh
                nh = m * rc
                cw = 0
                nw = nw//column
                if cw == 0:
                    cri = image[ch:nh,cw:nw]
                    ls.append(cri)
                n = nw
                for j in range(2,column+1):
                    cw = nw
                    nw = n*j
                    cri = image[ch:nh,cw:nw]
                    ls.append(cri)
            #i = i+1
        return ls 

    def avg_block_diff(self,block1,block2):
        ht1 , wd1 = block1.shape
        ht2 , wd2 = block2.shape

        a = np.mean(block1)
        b = np.mean(block2)
        avg = a - b
        return avg


    def align(self,refx,refy,x1,y1,img):
        rows, cols = img.shape
        if (refx==x1) and (refy==y1):
            #print('Same')
            return img
            #plt.imshow(img)
        else:
            difx= refx - x1
            dify = refy - y1
            if (dify>0):
                crop_img = img[0:rows,(cols-dify):cols]
                img = img[0:rows,0:(cols-dify)]
                im_h = cv2.hconcat([crop_img, img])
                #plt.imshow(im_h)
                img = im_h
                #return im_h
            elif (dify<0):
                dify = abs(dify)
                crop_img = img[0:rows,0:dify]
                img = img[0:rows,dify:cols]
                im_h = cv2.hconcat([img,crop_img])
                img = im_h
                #plt.imshow(im_h)
                #return im_h
            if (difx>0):
                crop_img = img[0:difx,0:cols]
                img = img[difx:rows,0:cols]
                im_v = cv2.vconcat([img,crop_img])
                img = im_v
                #plt.imshow(im_v)
                #return im_v
            elif (difx<0):
                difx = abs(difx)
                crop_img = img[(rows-difx):rows,0:cols]
                img = img[0:(rows-difx),0:cols]
                im_v = cv2.vconcat([crop_img, img])
                img = im_v
                #plt.imshow(im_v)
            return img

def main():

    obj = Image()

    excel_path_1 = '/home/office5/Desktop/TESTING/points_1.csv'
    df_1 = pd.read_csv(excel_path_1)

    excel_path_2 = '/home/office5/Desktop/TESTING/points_2.csv'
    df_2 = pd.read_csv(excel_path_2)

    # Aligninig image
    
    height,width = 4,3 
    path = '/home/office5/Desktop/TESTING/'
    refx, refy = df_1['midx'].iloc[0], df_1['midy'].iloc[0]

    # From first CSV file
    crped_img = []
    for i in range(len(df_1.index)):
        img = df_1['File name'].iloc[i]
        img = f'{path}/{img}'
        x1, y1 = df_1['midx'].iloc[i], df_1['midy'].iloc[i]
        img = cv2.imread(img,0)
        crped_img.append(obj.align(refx,refy,x1,y1,img))

    hei,wid = crped_img[0].shape
    master_img = np.zeros([hei, wid])

    N = len(crped_img)

    # Generate master image from first CSV file after image alignment
    for im in crped_img:
        imarr=np.array(im)
        master_img=master_img + imarr / N

    ac = []
    ac.append(obj.get_block(master_img,height,width))

    # From second CSV file and align image with respect to reference image
    lst_2 = []
    crped_img_2 = []
    for i in range(len(df_2.index)):
        img_2 = df_2['File name'].iloc[i]
        img_2 = f'{path}/{img_2}'
        img_2 = cv2.imread(img_2,0)
        crped_img_2.append(obj.align(refx,refy,x1,y1,img_2))
        each_blk_2 = obj.get_block(crped_img_2[i],height,width)
        lst_2.append(each_blk_2)


    # Get difference between master image and second csv file image after alignment
    Difference_lst_2 = []
    print('Here to get difference between master image and rest other image')
    for i in range(len(lst_2)):
        start = time.time()
        lst2 = []
        for j in range(len(lst_2[1])):
            lst2.append(obj.avg_block_diff(ac[0][j],lst_2[i][j]))
        Difference_lst_2.append(lst2)
        print(f"Difference between master_img and {df_2['File name'].iloc[i]}")
        end = time.time()
        print("time difference is ",end - start)
        print(lst2)
        print('\n')

    with open("train.csv", "w") as f:
        graphFileWriter = csv.writer(f)
        for i in range(len(df_1.index)):
            img = df_1['File name'].iloc[i]
            a = np.array(Difference_lst_2[i])
            a = np.reshape(a, (4, 3))
            f.write('\n' + img + '\n')
            graphFileWriter.writerows(a)

main()

Here to get difference between master image and rest other image
Difference between master_img and Anna12.bmp
time difference is  0.0656592845916748
[0.12717560502815672, 0.377740511294931, 0.12898056735028263, -0.016612366574889847, -0.16475957262015584, -0.09090795137967689, 0.17914981370833516, 0.3174829337574181, 0.05025447267725269, 0.023775310905591596, -0.055707855716320864, -0.056210245200105646]


Difference between master_img and Anna13.bmp
time difference is  0.055880069732666016
[0.15828557162993206, 0.07926902566120475, 0.02252528992680425, 0.04968164691867116, -0.02196012142519521, -0.07070170893203453, 0.10345048586864891, 0.21166908272357432, 0.08042563859427787, 0.07760163172997636, 0.14271737169372045, -0.013365794508644058]


Difference between master_img and Anna14.bmp
time difference is  0.05517268180847168
[0.03398993437832587, -0.12166631352381785, -0.21293693567066896, 0.12634232763832642, 0.237281558388176, 0.07669943902713783, 0.080262108955381, 0.036345694733

In [18]:
import pandas as pd
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import csv



class Image:

    def __init__(self) -> None:
        pass   
            
    def get_block(self,image,row,column):
        # img = cv2.imread(image,0)
        ht, wd = image.shape
        ch = 0 
        cw = 0
        nh = ht//row
        nw = wd//column
        ls = []
        m = nh

        if cw == 0:
            cri = image[ch:nh,cw:nw]
            # print(type(cri))
            ls.append(cri)
        n = nw
        for j in range(2,column+1):
            cw = nw
            nw = n*j
            cri = image[ch:nh,cw:nw]
            ls.append(cri) 
        for i in range(0,row):
            if i > 0:
                rc = i+1
                #print(i)
                ch = nh
                nh = m * rc
                cw = 0
                nw = nw//column
                if cw == 0:
                    cri = image[ch:nh,cw:nw]
                    ls.append(cri)
                n = nw
                for j in range(2,column+1):
                    cw = nw
                    nw = n*j
                    cri = image[ch:nh,cw:nw]
                    ls.append(cri)
            #i = i+1
        return ls 

    def avg_block_diff(self,block1,block2):
        ht1 , wd1 = block1.shape
        ht2 , wd2 = block2.shape

        a = np.mean(block1)
        b = np.mean(block2)
        avg = a - b
        return avg


    def align(self,refx,refy,x1,y1,img):
        rows, cols = img.shape
        if (refx==x1) and (refy==y1):
            #print('Same')
            return img
            #plt.imshow(img)
        else:
            difx= refx - x1
            dify = refy - y1
            if (dify>0):
                crop_img = img[0:rows,(cols-dify):cols]
                img = img[0:rows,0:(cols-dify)]
                im_h = cv2.hconcat([crop_img, img])
                #plt.imshow(im_h)
                img = im_h
                #return im_h
            elif (dify<0):
                dify = abs(dify)
                crop_img = img[0:rows,0:dify]
                img = img[0:rows,dify:cols]
                im_h = cv2.hconcat([img,crop_img])
                img = im_h
                #plt.imshow(im_h)
                #return im_h
            if (difx>0):
                crop_img = img[0:difx,0:cols]
                img = img[difx:rows,0:cols]
                im_v = cv2.vconcat([img,crop_img])
                img = im_v
                #plt.imshow(im_v)
                #return im_v
            elif (difx<0):
                difx = abs(difx)
                crop_img = img[(rows-difx):rows,0:cols]
                img = img[0:(rows-difx),0:cols]
                im_v = cv2.vconcat([crop_img, img])
                img = im_v
                #plt.imshow(im_v)
            return img

def main():

    obj = Image()

    excel_path_1 = '/home/office5/Desktop/TESTING/points_1.csv'
    df_1 = pd.read_csv(excel_path_1)

    excel_path_2 = '/home/office5/Desktop/TESTING/points_2.csv'
    df_2 = pd.read_csv(excel_path_2)

    # Aligninig image
    
    height,width = 4,3 
    path = '/home/office5/Desktop/TESTING/'
    refx, refy = df_1['midx'].iloc[0], df_1['midy'].iloc[0]

    # From first CSV file
    crped_img = []
    for i in range(len(df_1.index)):
        img = df_1['File name'].iloc[i]
        img = f'{path}/{img}'
        x1, y1 = df_1['midx'].iloc[i], df_1['midy'].iloc[i]
        img = cv2.imread(img,0)
        crped_img.append(obj.align(refx,refy,x1,y1,img))

    hei,wid = crped_img[0].shape
    master_img = np.zeros([hei, wid])

    N = len(crped_img)

    # Generate master image from first CSV file after image alignment
    for im in crped_img:
        imarr=np.array(im)
        master_img=master_img + imarr / N

    ac = []
    ac.append(obj.get_block(master_img,height,width))

    # From second CSV file and align image with respect to reference image
    lst_2 = []
    crped_img_2 = []
    for i in range(len(df_2.index)):
        img_2 = df_2['File name'].iloc[i]
        img_2 = f'{path}/{img_2}'
        img_2 = cv2.imread(img_2,0)
        crped_img_2.append(obj.align(refx,refy,x1,y1,img_2))
        each_blk_2 = obj.get_block(crped_img_2[i],height,width)
        lst_2.append(each_blk_2)


    # Get difference between master image and second csv file image after alignment
    Difference_lst_2 = []
    print('Here to get difference between master image and rest other image')
    for i in range(len(lst_2)):
        start = time.time()
        lst2 = []
        for j in range(len(lst_2[1])):
            lst2.append(obj.avg_block_diff(ac[0][j],lst_2[i][j]))
        Difference_lst_2.append(lst2)
        print(f"Difference between master_img and {df_2['File name'].iloc[i]}")
        end = time.time()
        print("time difference is ",end - start)
        print(lst2)
        print('\n')

    with open("train.csv", "w") as f:
        graphFileWriter = csv.writer(f)
        for i in range(len(df_1.index)):
            img = df_1['File name'].iloc[i]
            a = np.array(Difference_lst_2[i])
            a = np.reshape(a, (4, 3))
            f.write('\n' + img + '\n')
            graphFileWriter.writerows(a)

main()

Here to get difference between master image and rest other image
Difference between master_img and Anna12.bmp
time difference is  0.04831862449645996
[0.12717560502815672, 0.377740511294931, 0.12898056735028263, -0.016612366574889847, -0.16475957262015584, -0.09090795137967689, 0.17914981370833516, 0.3174829337574181, 0.05025447267725269, 0.023775310905591596, -0.055707855716320864, -0.056210245200105646]


Difference between master_img and Anna13.bmp
time difference is  0.04530692100524902
[0.15828557162993206, 0.07926902566120475, 0.02252528992680425, 0.04968164691867116, -0.02196012142519521, -0.07070170893203453, 0.10345048586864891, 0.21166908272357432, 0.08042563859427787, 0.07760163172997636, 0.14271737169372045, -0.013365794508644058]


Difference between master_img and Anna14.bmp
time difference is  0.04703211784362793
[0.03398993437832587, -0.12166631352381785, -0.21293693567066896, 0.12634232763832642, 0.237281558388176, 0.07669943902713783, 0.080262108955381, 0.036345694733

In [ ]:
import pandas as pd
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time



class Image:

    def __init__(self) -> None:
        pass


    def get_block(self,image,row,column):
        # img = cv2.imread(image,0)
        ht, wd = image.shape
        ch = 0 
        cw = 0
        nh = ht//row
        nw = wd//column
        ls = []
        m = nh

        if cw == 0:
            cri = image[ch:nh,cw:nw]
            # print(type(cri))
            ls.append(cri)
        n = nw
        for j in range(2,column+1):
            cw = nw
            nw = n*j
            cri = image[ch:nh,cw:nw]
            ls.append(cri) 
        for i in range(0,row):
            if i > 0:
                rc = i+1
                #print(i)
                ch = nh
                nh = m * rc
                cw = 0
                nw = nw//column
                if cw == 0:
                    cri = image[ch:nh,cw:nw]
                    ls.append(cri)
                n = nw
                for j in range(2,column+1):
                    cw = nw
                    nw = n*j
                    cri = image[ch:nh,cw:nw]
                    ls.append(cri)
            #i = i+1
        return ls 

    def avg_block_diff(self,block1,block2):
        ht1 , wd1 = block1.shape
        ht2 , wd2 = block2.shape

        a = np.mean(block1)
        b = np.mean(block2)
        avg = a - b
        return avg


    def align(self,refx,refy,x1,y1,image):
        img = cv2.imread(image,0)
        rows, cols = img.shape
        if (refx==x1) and (refy==y1):
            #print('Same')
            return img
            #plt.imshow(img)
        else:
            difx= refx - x1
            dify = refy - y1
            if (dify>0):
                crop_img = img[0:rows,(cols-dify):cols]
                img = img[0:rows,0:(cols-dify)]
                im_h = cv2.hconcat([crop_img, img])
                #plt.imshow(im_h)
                img = im_h
                #return im_h
            elif (dify<0):
                dify = abs(dify)
                crop_img = img[0:rows,0:dify]
                img = img[0:rows,dify:cols]
                im_h = cv2.hconcat([img,crop_img])
                img = im_h
                #plt.imshow(im_h)
                #return im_h
            if (difx>0):
                crop_img = img[0:difx,0:cols]
                img = img[difx:rows,0:cols]
                im_v = cv2.vconcat([img,crop_img])
                img = im_v
                #plt.imshow(im_v)
                #return im_v
            elif (difx<0):
                difx = abs(difx)
                crop_img = img[(rows-difx):rows,0:cols]
                img = img[0:(rows-difx),0:cols]
                im_v = cv2.vconcat([crop_img, img])
                img = im_v
                #plt.imshow(im_v)
            return img

def main():

    obj = Image()

    excel_path = '/home/office5/Desktop/TESTING/points.csv'
    df = pd.read_csv(excel_path)

    # Aligninig image
    print('Here time Used for alignment')

    start2 = time.time()
    crped_img = []
    height,width = 4,3 
    lst=[]
    path = '/home/office5/Desktop/TESTING/'
    refx, refy = df['midx'].iloc[0], df['midy'].iloc[0]
    for i in range(len(df.index)):
        img = df['File name'].iloc[i]
        img = f'{path}/{img}'
        x1, y1 = df['midx'].iloc[i], df['midy'].iloc[i]
        crped_img.append(obj.align(refx,refy,x1,y1,img))
        each_blk = obj.get_block(crped_img[i],height,width)
        lst.append(each_blk)

    end2 = time.time()
    print("time for generating alignment image ",end2 - start2)

    print("Here time used for generating master Image")
    start1 = time.time()

    hei,wid = crped_img[0].shape
    master_img = np.zeros([hei, wid])
    for i in range(hei):
        for j in range(wid):
            master_img[i,j] = 0
            for k in range(len(df.index)):
                master_img[i,j] += (int(crped_img[k][i,j]))
            master_img[i,j] = master_img[i,j] / len(crped_img)

    end1 = time.time()
    print("time for generating master image ",end1 - start1)
    
    ac = []
    ac.append(obj.get_block(master_img,height,width))

    print('Here to get difference between master image and rest other image')
    for i in range(len(lst)):
        start = time.time()
        lst1 = []
        for j in range(len(lst[1])):
            lst1.append(obj.avg_block_diff(ac[0][j],lst[i][j]))
        print(f"Difference between master_img and {df['File name'].iloc[i]}")
        end = time.time()
        print("time difference is ",end - start)
        print(lst1)
        print('\n')

main()